In [1]:
# Import packages
import numpy as np
import pandas as pd
import os
import datetime as dt

In [2]:
# Set directories
data_dir = os.getcwd()
which_year = '2018'
if which_year == '2018':
    filenames = ['indego_trips_2018_q1', 'indego_trips_2018_q2',
                 'indego_trips_2018_q3']
elif which_year == '2017': 
    filenames = ['indego_trips_2017_q1', 'indego_trips_2017_q2',
             'indego_trips_2017_q3', 'indego_trips_2017_q4']
elif which_year == '2016': 
    filenames = ['indego_trips_2016_q1', 'indego_trips_2016_q2',
             'indego_trips_2016_q3', 'indego_trips_2016_q4']
column_names = ['trip_id', 'duration', 'start_time', 'end_time','start_station','start_lat','start_lon', 
                'end_station','end_lat','end_lon', 'bike_id', 'plan_duration', 'trip_route_category', 'passholder_type']
file_format = '.csv'

In [3]:
# Concatinate all quarters for a given year into a single data frame
df = pd.DataFrame(columns = column_names)
for x in range(len(filenames)): 
    current_file = os.path.join(data_dir + '/' + filenames[x] + file_format)
    temp = pd.read_csv(current_file)
    df = df.append(temp)

In [4]:
# Identify potentially anomalous trips (based on length) and drop them
maxDuration = df["duration"].mean() + df["duration"].std()*2
minDuration  = 1
numDroppedTop = (len(df[df['duration']>=maxDuration]))/(len(df[df['duration']<maxDuration]))
numDroppedBottom = (len(df[df['duration']<=minDuration]))/(len(df[df['duration']>minDuration]))
# MINOR: make this a nice output
print(numDroppedTop)
print(numDroppedBottom)

0.014099142624886886
0.01875156487740991


In [5]:
# extract date and hour from the start dat
pd.to_datetime(df['start_time'])
df['start_time_date'] = pd.to_datetime(df['start_time']).dt.to_period('D')
df['start_time_hour'] = pd.DatetimeIndex(df['start_time']).hour

In [6]:
#group trips by station, then get date and hour
station_df = df.groupby(['start_station', 'start_time_date', 'start_time_hour']).count()

In [7]:
new_dataframe = 'AggregatedTrips' + which_year + '.csv'
station_df.to_csv(new_dataframe)